In [ ]:
using Plots
using FFTW

In [ ]:
L = 100
N = 1000
dt = 1e-2

dx = L/N
dk = 2*π / L

x = dx .* collect(0:N-1)
k = dk .* collect(0:N÷2)

k² = k.^2
c = -k.^4
cdt = c .* dt
k1 = exp.(cdt)
k2 = dt * ( exp.(cdt) .- 1 ) ./ cdt
k2[1] = 1.

F = plan_rfft(x)
B = plan_irfft(im * k, N)

α = .4


function g(φ1, φ2)
    urφ² = 1 .- (φ1.^2 .+ φ2.^2)
    g1 = k² .* (F * (urφ².*φ1 .+ α .* φ2 ))
    g2 = k² .* (F * (urφ².*φ2 .- α .* φ1 ))
    return g1, g2
end

In [ ]:
function antialias!(F)
    F[end-(N÷6)+1:end] .= 0.
end

function etd!(Fφ1, Fφ2, φ1, φ2)
    g1, g2 = g(φ1, φ2)
    Fφ1 .= k1 .* Fφ1 + k2 .* g1
    Fφ2 .= k1 .* Fφ2 + k2 .* g2
    antialias!(Fφ1)
    antialias!(Fφ2)
    φ1 .= B*Fφ1
    φ2 .= B*Fφ2
end

In [ ]:
v = .3
A = 1e-1
φ1 = v * ones(N) + A* cos.((2π/L).*x)
φ2 = A * sin.((2π/L).*x); 
Fφ1, Fφ2 = F*φ1, F*φ2;

In [ ]:
M = Int(1e5)
frames = 1000
n = M ÷ frames

φt = Array{Float64}(undef, (frames, 2, N))
φt[1, 1, :] .= φ1
φt[1, 2, :] .= φ2;

In [ ]:
for i in 2:frames
    for j in 1:n
        etd!(Fφ1, Fφ2, φ1, φ2)
        t = i*n + j
        if t%(M÷10) == 0
            print(100*t/M,"% \n")
            @assert !any(isnan.(Fφ1))
        end
    end
    φt[i, 1, :] .= φ1
    φt[i, 2, :] .= φ2
end

In [ ]:
anim = @animate for i in 1:frames
    plot()
    plot!(x, φt[i, 1, :])
    plot!(x, φt[i, 2, :])
    plot!(;yrange=(-1, 1))
end;
gif(anim, "nrch.mp4", fps = 15)